In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import imageio
import matplotlib.pyplot as plt
import os
import sklearn
import cv2
import zipfile

print(os.listdir("../input"))
with zipfile.ZipFile("../input/train.zip","r") as z:
    z.extractall(".")
    
with zipfile.ZipFile("../input/test1.zip","r") as z:
    z.extractall(".")

## Define the datasets (Dogs vs. Cats from Kaggle)

In [2]:
dataset_path = "/kaggle/working/"
train_dataset = "/kaggle/working/train"
files = os.listdir(train_dataset)

In [3]:
# Training Dataset
train_x = []
train_y = []

# Iterate through files in training dataset
for f in files:
  # Get label/animal from filename - ex. cat.0.jpg, label is before .
  label = f.split(".")[0]
  filepath = train_dataset + "/" + f                  # image filepath
  data = imageio.imread(filepath, as_gray = True)     # read the specified image (in greyscale)
  data_arr = cv2.resize(data, dsize = (80, 80))       # resize the image to 90x90
  # add image data and label to respective arrays
  train_x.append(data_arr)
  train_y.append(label)

In [4]:
# Training Data
print(len(train_x))
print(len(train_y))

## Normalize/Scale Data and Change Dimensions


In [5]:
X_train = tf.keras.utils.normalize(train_x, axis=1) 

# need to reshape the data as keras needs 4D datasets, and ours are 3D right now
X_train = np.expand_dims(X_train, axis=-1)

# new reshaped dataset
print(X_train.shape)

## Label Encoding (Emotions -> Numbers)

Need to change the string values of animals (cat or dog) into numbers so the CNN can properly predict them based on classes/labels.

In [6]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
Y_train = lb.fit_transform(train_y)
Y_train

## Building the Model

In [7]:
import tensorflow.keras as keras

# build the model
model = keras.Sequential()

# add the layers

# hidden layers
model.add(keras.layers.Conv2D(128, 3, input_shape=(80, 80, 1), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
tf.keras.layers.Dropout(0.30)

model.add(keras.layers.Conv2D(256, 3, activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
tf.keras.layers.Dropout(0.40)

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
history = model.fit(X_train, Y_train, epochs=10, validation_split=0.2, batch_size=32)

In [9]:
# Save the model
model.save('model.h5')
json_model = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(json_model)

print("Saved model!")

In [10]:
# Display model stats

# Model Accuracy
plt.title('CNN Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Model Loss
plt.title('CNN Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Test Dataset Preparation

In [11]:
test_data = []
test_data_image = []
test_dataset = "/kaggle/working/test1"
files = os.listdir(test_dataset)

for f in files:
  id = f.split(".")[0]
  filepath = test_dataset + "/" + f
  data = imageio.imread(filepath, as_gray = True)     # read the specified image (in greyscale)
  data_arr = cv2.resize(data, dsize = (80, 80))       # resize the image to 90x90
  test_data_image.append(data_arr)


In [12]:
test_data = tf.keras.utils.normalize(test_data_image, axis=1) 

# need to reshape the data as keras needs 4D datasets, and ours are 3D right now
test_data = np.expand_dims(test_data, axis=-1)
# new reshaped dataset
print(test_data.shape)

labels = list(lb.classes_)
labels

In [13]:
predictions = model.predict(test_data)

# to get as classes (since we used sigmoid, will have to round)
classes = []
for p in predictions:
  classes.append(round(p[0]))

In [18]:
import random
import matplotlib.pyplot as plt

size = len(test_data_image)

image = random.randint(0, size)
plt.imshow(test_data_image[image], cmap="gray")
plt.show()

model_prediction = int(classes[image])

print("Model prediction - class:", model_prediction, "which is a", labels[model_prediction])